# Herbert - Polish RoBERT'a model 


## Prepare the data.

* Wikipedia, Link: 
* Oscar
* Books, Polish "Wolne lektury"



Get raw text from different sources and concat in one big data file. 

Usefull shell commands:


Move files to another directory,  where isbn's are in the file list (wolne lektury non polish)

```
cat wolne_lektury_non_polish_isbn.txt | xargs -I{} sh -c "mv *'{}'* ./non_polish/;" 
```


Cat all text files and instert new line between each text

```sh
find *content.txt | xargs -I{} sh -c "cat '{}'; echo ''" > corpus_[type]_[date].txt

```

Take 11768022 first lines form splited wikipedia file
```
head -11768022 corpus_wiki_books_non_word_tokenized_2020-02-13.txt > corpus_wiki_lines_non_tok_2020-02-13.txt
```


In [2]:
import csv
import sys
import datetime as dt
import os
from pathlib import Path
import re
import tqdm as tq
from tqdm.notebook import tqdm
import mmap

print("import")

import


In [3]:
# utils functions


def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

Read wikipedia csv file and save in txt.

Run only once, just for generating file!!!

In [4]:

wikipedia_path='./data/wikipedia_2020-02-01.csv'
output_path='./data/corpus_wikipedia_2020-02-01.txt'

csv.field_size_limit(sys.maxsize)

t0=dt.datetime.now()
with open(output_path, 'w+') as output_file:
    with open(wikipedia_path) as csv_file:
        csv_reader = csv.DictReader(csv_file)

        for article in csv_reader:
            text = article['CleanText']

            output_file.write(text)
            # put new line of the end of the article
            output_file.write('\n')   
t1=dt.datetime.now()  
print(f'Done. Takes={t1-t0}')   

Read book dataset and normalize line splitiing. The textfile has '\n' in middle of the sentence. It is not necessary if you have proper file.

Input: concatenated book textfile
Output: file with removed new lines in the middle of the sentence. 

Run once!!!


In [5]:


input_path='./data/corpus_books_2020_02_24.txt'

p = Path(input_path)
print(p.with_suffix(''))
output_path = f"{p.with_suffix('')}_fix.txt"

print(output_path)

# reg = re.compile('(?<!\.)(\n)(?=[a-zA-ZąćęłńóśźżĄĆĘŁŃÓŚŹŻ])',re.MULTILINE )
# reg = re.compile('(?<!\.)(\n)(?=[a-ząćęłńóśźż])',re.MULTILINE )
#reg = re.compile('(?<!\.\n)(\n)(?=[a-ząćęłńóśźż])',re.MULTILINE )


# remove line breaks in the middle of the sentence
reg = re.compile('(?<=[A-Za-ząćęłńóśźż,—-])(?<!\.)(\n)(?=[a-ząćęłńóśźż])',re.MULTILINE )
#replace it by space
rep_lines=' '

# replace many dots in lines with one dot and line break
reg_dots = re.compile('(\.*\n){2,}',re.MULTILINE)
rep_dots='.\n'

# https://pymotw.com/3/mmap/#regular-expressions

line_buff = 10007# 10007 # this is prime number, next is  10009  10037 
N = 0 

def save_buffer2file(output_file, text):

    # proces and write
    replace_text = reg.sub(rep_lines, text)
    #clean lines with only one character (dots, etc)
    replace_text = reg_dots.sub(rep_dots, replace_text)

    output_file.write(replace_text)



t0=dt.datetime.now()
with open(output_path, 'w+') as output_file:
    with open(input_path) as f:

        text=''
        for line in tqdm(f,total=get_num_lines(input_path)):
            # get block of file (line_buff) lines and replace 
            if N<line_buff:
                # glue lines
                text+=line
                N+=1
            else:
                save_buffer2file(output_file, text)
                
                text=''
                N=0
                
        # for the rest of the file proces and write
        if N>0:
            save_buffer2file(output_file, text)
            

t1=dt.datetime.now()  
print(f'Done. Takes={t1-t0}')  

data/corpus_books_2020_02_24
data/corpus_books_2020_02_24_fix.txt


## Create vocabulary


### Prepare data for vocab

Create files for training vocabulary. One sentence per line, use polish sentence tokenizer. 
Sentence piece model is capable of handling around 12_000_000 sentences, so more sentences is not necessary.





In [ ]:
import nltk

#nltk.download()
nltk.download('punkt')




Add necessary abbreviations to NLTK for wikipedia text

In [ ]:


extra_abbreviations = ['ps',  'inc', 'Corp', 'Ltd', 'Co', 'pkt', 'Dz.Ap', 'Jr', 'jr', 'sp', 'Sp', 'poj',  'pseud', 'krypt', 'sygn', 'Dz.U', 'ws', 'itd', 'np', 'sanskryt', 'nr', 'gł', 'Takht', 'tzw', 't.zw', 'ewan', 'tyt', 'oryg', 't.j', 'vs', 'l.mn', 'l.poj' ]

position_abbrev = ['Ks', 'Abp', 'abp','bp','dr', 'kard', 'mgr', 'prof', 'zwycz', 'hab', 'arch', 'arch.kraj', 'B.Sc', 'Ph.D', 'lek', 'med', 'n.med', 'bł', 'św', 'hr', 'dziek' ]

roman_abbrev= [] #['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'XI', 'XII', 'XII','XIV','XV','XVI', 'XVII', 'XVIII','XIX', 'XX', 'XXI' ]

quantity_abbrev = [ 'mln', 'obr./min','km/godz', 'godz', 'egz', 'ha', 'j.m', 'cal', 'obj', 'alk', 'wag' ] # not added: tys.

actions_abbrev = ['tłum','tlum','zob','wym', 'pot', 'ww', 'ogł', 'wyd', 'min', 'm.i', 'm.in', 'in', 'im','muz','tj', 'dot', 'wsp', 'właść', 'właśc', 'przedr', 'czyt', 'proj', 'dosł', 'hist', 'daw', 'zwł', 'zaw' ]

place_abbrev = ['Śl', 'płd', 'geogr']

lang_abbrev = ['jęz','fr','franc', 'ukr', 'ang', 'gr', 'hebr', 'czes', 'pol', 'niem', 'arab', 'egip', 'hiszp', 'jap', 'chin', 'kor', 'tyb', 'wiet', 'sum']

military_abbrev = ['kpt', 'kpr', 'obs', 'pil', 'mjr','płk', 'dypl', 'pp', 'gw', 'dyw', 'bryg', 'ppłk', 'mar', 'marsz', 'rez', 'ppor', 'DPanc', 'BPanc', 'DKaw', 'p.uł']

extra_abbreviations= extra_abbreviations + position_abbrev + roman_abbrev + quantity_abbrev + place_abbrev + actions_abbrev + place_abbrev + lang_abbrev+military_abbrev

sentence_tokenizer = nltk.data.load('tokenizers/punkt/polish.pickle')

abbrev_set_curr = sentence_tokenizer._params.abbrev_types.copy()
#update abbrev
sentence_tokenizer._params.abbrev_types.update(extra_abbreviations)
abbrev_set_upd = sentence_tokenizer._params.abbrev_types.copy()

print(f'\ncurrent abbrev={abbrev_set_curr}')
sentence_tokenizer._params.abbrev_types.update(extra_abbreviations)
print(f'\nupdated abbrev={abbrev_set_upd}')

print(f'symmetric diff= {abbrev_set_upd.symmetric_difference(abbrev_set_curr)}')

print(f'Updatad diff= {abbrev_set_upd.difference(abbrev_set_curr)}')
print(f'Curr diff= {abbrev_set_curr.difference(abbrev_set_upd)}')






In [ ]:
files_to_proces = [
    './data/corpus_wikipedia_2020-02-01.txt',
    './data/corpus_books_2020_02_24.txt'
]
files_to_proces = [
    './data/corpus_wikipedia_2020-02-01.txt',
]
files_to_proces = [
    './data/corpus_books_2020_02_24_fix.txt'
]

from nltk.tokenize import sent_tokenize

for input_file in files_to_proces:
    print(input_file)
    t0=dt.datetime.now()

    p = Path(input_file)
    output_path = f"{p.with_suffix('')}_lines.txt"
    text=''
    with open(output_path, 'w+') as output_file:
        with open(input_file) as f:
            for line in tqdm(f,total=get_num_lines(input_file)):

                # get block of text to new line which splits ariticles
                text+=line
                if line.strip() == '':

                    sentences = sentence_tokenizer.tokenize(text)

                    file_content = ''
                    for sentence in sentences:
                        file_content += sentence.strip()
                        file_content+='\n'
                    output_file.write(file_content)
                    
                    output_file.write('\n')
                    text=''

                ## old way
                # if line.strip() == '':
                #     output_file.write('\n')
                #     continue;

                # sentences = sentence_tokenizer.tokenize(line)
                
                # text = ''
                # for sentence in sentences:
                #     text += sentence.strip()
                #     text+='\n'
                # output_file.write(text)
                
    t1=dt.datetime.now()  
    print(f'Done. Takes={t1-t0}')   

### Train the BPE sentencepiece model

We used the [sentencepiece](https://github.com/google/sentencepiece)

We get: 

- 8 mln lines from wiki 
- 12 mln lines from books

This can be done in two ways:
- as a python module
- as a command line 


If you want to use it as a commandline you should [build the C++ version from source](https://github.com/google/sentencepiece#c-from-source)



```
head -8M corpus_wikipedia_2020-02-01_lines.txt > corpus_wikipedia_2020-02-01_lines_8M.txt
head -12M corpus_books_2020_02_13_fix_lines.txt > corpus_books_2020_02_13_fix_lines_12M.txt
```


Train using commandline

```
spm_train \
    --input='./data/corpus_wikipedia_2020-02-01_lines.txt' \
    --max_sentence_length= [ max length of a sentence you accept ]\
    --model_prefix='.data/wikipedia.spm.bpe \
    --vocab_size=32000 \
    --model_type=bpe \
    --shuffle_input_sentence=true \
    --input_sentence_size=10000000 \
    --bos_id=0 --eos_id=1 --pad_id=2 --unk_id=3
```


In [1]:
import sentencepiece as spm


In [ ]:
max_vocab_size = 32000
model_type = "bpe"  
iss = 12_000_000

lower_vocab = False
txt_lower = "lower" if lower_vocab else "upper"
tok_model = f"wikibooks_{txt_lower}_voc_{max_vocab_size}_sent{iss}"

data_file = './data/corpus_wikipedia_2020-02-01_lines.txt'

tok_model = os.path.abspath(f"./data/{tok_model}")

piece_options = ' --bos_id=0 --eos_id=1 --pad_id=2 --unk_id=3 --shuffle_input_sentence=true'


cmd = f"--input={data_file} --model_prefix={tok_model} --model_type={model_type} --num_threads=4 --vocab_size={vocab_size}  --input_sentence_size={iss}"+ piece_options
print(cmd)


